In [1]:
# import libraries
# lib for pre trained model 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
# lib for layers
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
# lib for connecting pretrained and created model
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
#lib for compile
from tensorflow.keras.optimizers import Adam
# lib for pre processing
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
# lib for spliting i/p and o/p
from sklearn.model_selection import train_test_split
# lib for metrics
from sklearn.metrics import classification_report
# other lib
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

C:\Users\User\anaconda3\envs\facemask\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\anaconda3\envs\facemask\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\anaconda3\envs\facemask\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\anaconda3\envs\facemask\lib\site-packages\tensorflow\

In [2]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
# to assign the dataset path
ap.add_argument("-d", "--dataset", type=str, help="path to input dataset")
# to assign for output
ap.add_argument("-p", "--plot", type=str, default="plot.png",help="path to output loss/accuracy plot")
# for model
ap.add_argument("-m", "--model", type=str,default="mask_detectorch.model",help="path to output face mask detector model")
args, unknown = ap.parse_known_args()

In [3]:
# get the list of images in the dataset directory
print("[INFO] loading images...")
imagePaths = list(paths.list_images(r'S:/AI_RELATED/AI_PROJECTS/Face_Mask_Detection/dataset'))
data = []
labels = []

# Loop over image paths
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]

    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)

    data.append(image)
    labels.append(label)

# Convert to numpy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

# Split BEFORE encoding labels
(trainX, testX, trainY_raw, testY_raw) = train_test_split(
    data, labels, test_size=0.20, stratify=labels, random_state=42)

# Perform label binarization AFTER split
lb = LabelBinarizer()
trainY = to_categorical(lb.fit_transform(trainY_raw))
testY = to_categorical(lb.transform(testY_raw))

print(f"[INFO] Training samples: {trainX.shape[0]}, Testing samples: {testX.shape[0]}")



[INFO] loading images...


C:\Users\User\anaconda3\envs\facemask\lib\site-packages\PIL\Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


[INFO] Training samples: 3076, Testing samples: 769


In [4]:
from collections import Counter

# Count the number of samples per class
label_counts = Counter(labels)
print("[INFO] Class distribution:", label_counts)


[INFO] Class distribution: Counter({'without_mask': 1929, 'with_mask': 1916})


In [5]:
#construct the trianing image generator for data augementation

aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range= 0.2,
    shear_range= 0.15,
    horizontal_flip= True,
    fill_mode= 'nearest'
    )

In [6]:
# load the pre trained  model
basemodel = MobileNetV2(weights ='imagenet',include_top = False,
                        input_tensor = Input(shape = (224,224,3)))

# develope a head model to place in the top of the model
headmodel = basemodel.output
headmodel = AveragePooling2D(pool_size =(7,7))(headmodel)
headmodel = Flatten(name = 'flatten')(headmodel)
headmodel = Dense(128,activation ='relu')(headmodel)
headmodel = Dropout(0.5)(headmodel)
headmodel = Dense(2,activation ='softmax')(headmodel)

# building actual model by combining basemodel and headmodel of FC and that will be the training model
model = Model(inputs =basemodel.input,outputs = headmodel)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


C:\Users\User\anaconda3\envs\facemask\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [7]:
# loop all over layers in the basemodel and freeze it to stop the update

# mandatory step in all pre trained model program

for layer in basemodel.layers:
    layer.trainable =False

In [8]:
# compile model

print("[INFO] Compiling model....")
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32
opt = Adam(learning_rate=INIT_LR)
model.compile(loss="binary_crossentropy",optimizer =opt,
              metrics =["accuracy"])

[INFO] Compiling model....


In [9]:
# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)


[INFO] training head...
Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
96/96 [==============================] - 411s 4s/step - loss: 0.5888 - acc: 0.7053 - val_loss: 0.4342 - val_acc: 0.8270
Epoch 2/20
96/96 [==============================] - 447s 5s/step - loss: 0.3067 - acc: 0.8643 - val_loss: 0.3146 - val_acc: 0.8804
Epoch 3/20
96/96 [==============================] - 461s 5s/step - loss: 0.2489 - acc: 0.8985 - val_loss: 0.4438 - val_acc: 0.8257
Epoch 4/20
96/96 [==============================] - 466s 5s/step - loss: 0.1997 - acc: 0.9284 - val_loss: 0.3480 - val_acc: 0.8622
Epoch 5/20
96/96 [==============================] - 467s 5s/step - loss: 0.1751 - acc: 0.9336 - val_loss: 0.2817 - val_acc: 0.8973
Epoch 6/20
96/96 [==============================] - 442s 5s/step - loss: 0.1706 - acc: 0.9340 - val_loss: 0.3245 - val_acc: 0.8791
Epoch 7/20
96/96 [==============================] - 431s 4s/step - loss: 0.1482 - acc: 0.9468 - 

In [11]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

   with_mask       0.99      0.86      0.92       383
without_mask       0.87      0.99      0.93       386

    accuracy                           0.92       769
   macro avg       0.93      0.92      0.92       769
weighted avg       0.93      0.92      0.92       769



In [13]:

#Save the model
print("[INFO] saving the mask detector model...")
model.save('mask_detector.h5')


[INFO] saving the mask detector model...
